In [2]:
import pandas as pd
import re
from transformers import T5Tokenizer
import nltk
nltk.download('punkt')

2023-03-22 09:49:44.749029: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 09:49:45.382285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

True

In [41]:
df = pd.read_excel('../WOT_extractive_original.xlsx').drop(columns=['Unnamed: 0']) 
df['input'] = df['input'].apply(lambda x: str(x).replace('|', '| ')) 
#df['input'] = df['input'].apply(lambda x: str(x).replace('\n', '\n '))
df['input'] = df['input'].apply(lambda x: str(x).replace(';', '; ')) 
df['output'] = df['output'].apply(lambda x: str(x).replace('|', '| ')) 
df['output'] = df['output'].apply(lambda x: str(x).replace('\\n', '\n')) 
df['output'] = df['output'].apply(lambda x: str(x).replace(';', '; ')) 

In [42]:
print(df.loc[674, 'output'])
df.loc[674, 'output'] = 'use a dry cloth to soak up any liquid from the vomit'

use a dry cloth to soak up any liquid from the vomi


In [43]:
model_name = "allenai/unifiedqa-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

def compare(a, b):
    for i in range(len(b)):
        if a[i] != b[i]:
            return False
    return True

def get_trunc_instructions(instructions, answer, available_tokens):
    if compare(answer, [148, 33, 612]):
        if len(instructions) < available_tokens:
            return instructions
        return instructions[-available_tokens:]
    if compare(answer, [4273]):
        if len(instructions) < available_tokens:
            return instructions
        return instructions[-available_tokens:]
    if compare(answer, [150]):
        if len(instructions) < available_tokens:
            return instructions
        return instructions[-available_tokens:]
    if compare(answer, [766, 8, 2912, 19, 344, 9455, 11, 3, 4013, 3, 1956, 89, 41, 2884, 11, 944, 3, 1956, 75, 61]):
        answer = [766, 8, 2912, 19, 344, 9455, 11, 3, 4013, 3, 1956, 89, 41, 2884, 11, 944, 3, 1956, 75, 137]
    if compare(answer, [314, 716, 41, 127, 1200, 6, 3, 99, 25, 114, 61]):
        answer = [314, 716, 41, 127, 1200, 6, 3, 99, 25, 114, 201]
    if compare(answer, [3, 99, 338, 5726, 3, 11049, 3321, 7, 6, 18874, 16, 387]):
        answer = [41, 99, 338, 5726, 3, 11049, 3321, 7, 6, 18874, 16, 387]
    if compare(answer, [3, 20987, 371, 3224, 2912]):
        answer = [41, 20987, 371, 3224, 2912]
    if compare(answer, [6273, 3380, 49, 342, 21, 44, 709, 505, 716, 41, 127, 95, 12, 204, 477, 61]):
        answer = [6273, 3380, 49, 342, 21, 44, 709, 505, 716, 41, 127, 95, 12, 204, 477, 137]
    
    try:
        answer = answer[:-1]
        position = [(i, i+len(answer)) for i in range(len(instructions)) if compare(instructions[i:i+len(answer)],answer)]
        start = position[0][0]
        end = position[0][1]
    except:
        print(tokenizer.decode(answer))
        print(tokenizer.decode(instructions))
        print(answer)
        print(instructions)
        
    
    before = True
    
    while end-start < available_tokens:
        if before:
            before = False
            start -= 10
        else:
            before = True
            end += 10
            
        start = max(start, 0)
        end = min(len(instructions), end)
        
        if start==0 and end ==len(instructions):
            return instructions[start:end]
    
    while end-start < available_tokens:
        if before:
            before = False
            start -= 1
        else:
            before = True
            end += 1
            
        start = max(start, 0)
        end = min(len(instructions), end)
        
        if start==0 and end ==len(instructions):
            return instructions[start:end]
        
    return instructions[start:end]
        
    

df['tokens_taken'] = df.apply(lambda row: len(tokenizer.encode(f"{row['question']}SPLITInstructions: \n\nHistory: {row['history']})")), axis=1)
df['encoded_instructions'] = df.apply(lambda row: tokenizer.encode(row['input'][len(f"{row['question']}SPLITInstructions: ") : len(row['input']) - len(f"\n\nHistory: {row['history']})")]), axis=1)
df['trunc_instructions'] = df.apply(lambda row: get_trunc_instructions(row['encoded_instructions'], tokenizer.encode(row['output']), 512 - row['tokens_taken']), axis=1)
df['trunc_instructions'] = df['trunc_instructions'].apply(lambda x: tokenizer.decode(x))
df['input_context_selected'] = df.apply(lambda row: f"{row['question']}SPLITInstructions:{row['trunc_instructions']}\n \n History: {row['history']}", axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors


In [44]:
df.to_excel('../WOT_extractive_final.xlsx')
